In [336]:

import time
import random
import numpy as np
import networkx as nx
import logging
from itertools import combinations
import matplotlib.pyplot as plt
from icecream import ic

===========================================================================================
### Definizione del Problema
===========================================================================================

In [337]:
class Problem:
    _graph: nx.Graph
    _alpha: float
    _beta: float

    def __init__(
        self,
        num_cities: int,
        *,
        alpha: float = 1.0,
        beta: float = 1.0,
        density: float = 0.5,
        seed: int = 42,
    ):
        rng = np.random.default_rng(seed)
        self._alpha = alpha
        self._beta = beta
        cities = rng.random(size=(num_cities, 2))
        cities[0, 0] = cities[0, 1] = 0.5

        self._graph = nx.Graph()
        self._graph.add_node(0, pos=(cities[0, 0], cities[0, 1]), gold=0)
        for c in range(1, num_cities):
            self._graph.add_node(c, pos=(cities[c, 0], cities[c, 1]), gold=(1 + 999 * rng.random()))

        tmp = cities[:, np.newaxis, :] - cities[np.newaxis, :, :]
        d = np.sqrt(np.sum(np.square(tmp), axis=-1))
        for c1, c2 in combinations(range(num_cities), 2):
            if rng.random() < density or c2 == c1 + 1:
                self._graph.add_edge(c1, c2, dist=d[c1, c2])

        assert nx.is_connected(self._graph)

    @property
    def graph(self) -> nx.Graph:
        return nx.Graph(self._graph)

    @property
    def alpha(self):
        return self._alpha

    @property
    def beta(self):
        return self._beta

    def cost(self, path, weight):
        dist = nx.path_weight(self._graph, path, weight='dist')
        return dist + (self._alpha * dist * weight) ** self._beta

    def baseline(self):
        total_cost = 0
        for dest, path in nx.single_source_dijkstra_path(
            self._graph, source=0, weight='dist'
        ).items():
            cost = 0
            for c1, c2 in zip(path, path[1:]):
                cost += self.cost([c1, c2], 0)
                cost += self.cost([c1, c2], self._graph.nodes[dest]['gold'])
            logging.debug(
                f"dummy_solution: go to {dest} ({' > '.join(str(n) for n in path)} ({cost})"
            )
            total_cost += cost
        return total_cost

    def plot(self):
        plt.figure(figsize=(10, 10))
        pos = nx.get_node_attributes(self._graph, 'pos')
        size = [100] + [self._graph.nodes[n]['gold'] for n in range(1, len(self._graph))]
        color = ['red'] + ['lightblue'] * (len(self._graph) - 1)
        return nx.draw(self._graph, pos, with_labels=True, node_color=color, node_size=size)

===========================================================================================
### Matrice delle distanze
===========================================================================================

In [338]:
def get_distance_matrix(problem:Problem) -> np.ndarray:
    graph = problem._graph
    n = len(graph.nodes)

    dist_matrix = np.zeros((n, n))
    all_pairs = nx.all_pairs_dijkstra_path_length(graph, weight="dist")

    for src, targets in all_pairs:
        for tgt, d in targets.items():
            dist_matrix[src][tgt] = d

    return dist_matrix

In [339]:
problem = Problem(num_cities=100, alpha=1, beta=1, density=0.5, seed=42)
DISTANCE_MATRIX = get_distance_matrix(problem)

===========================================================================================
### Funzione di costo di una soluzione
===========================================================================================

In [340]:
def get_cost(problem: Problem, solution: list[int]) -> float:
    total_cost = 0.0
    transported_gold = 0.0

    path = [0] + solution + [0]  # start and end at depot

    empty_nodes = set() # nodes where we have picked up all the gold

    for actual_node_i in range(len(path) - 1):
        start_node = path[actual_node_i]
        destination_node = path[actual_node_i + 1]

        if start_node == 0: # if we are in the depot
            transported_gold = 0.0
        elif start_node not in empty_nodes:
            gold = problem._graph.nodes[start_node]['gold']
            transported_gold += gold
            empty_nodes.add(start_node)
            segment_cost = DISTANCE_MATRIX[start_node][destination_node] + (problem._alpha * DISTANCE_MATRIX[start_node][destination_node] * transported_gold) ** problem._beta
            total_cost += segment_cost
    return total_cost

===========================================================================================
### Algoritmi di Hill Climbing a supporto dell'inizializzazione della popolazione
===========================================================================================

In [341]:
def get_initial_solution(nodes: list[int]) -> list[int]:
    unvisited_nodes = set(nodes)
    unvisited_nodes.remove(0)
    
    current_node = 0
    path = [] 
    
    while unvisited_nodes:
        candidates = []
        for node in unvisited_nodes:
            distance = DISTANCE_MATRIX[current_node][node]
            candidates.append((node, distance))
        candidates.sort(key=lambda x: x[1])
        k = min(5, len(candidates))
        top_k_candidates = candidates[:k]
        chosen_node, _ = random.choice(top_k_candidates)
        path.append(chosen_node)
        unvisited_nodes.remove(chosen_node)
        current_node = chosen_node
    return path

$\beta$ < 1

In [342]:
def tweak_0(solution: list[int]) -> list[int]:
    path = solution.copy()
    path_length = len(path)
    i = random.randint(0, path_length - 2)
    j = random.randint(i + 1, path_length - 1)
    path[i:j+1] = path[i:j+1][::-1]
    return path



def hc_beta_less_than_1(problem:Problem, max_iterations:int)-> list[int]:
    current_solution = get_initial_solution(problem._graph.nodes)
    current_cost = get_cost(problem, current_solution)
    
    #print(f"Starting Cost: {current_cost}")

    for i in range(max_iterations):
        #print(f"Iteration {i}: Actual Cost = {current_cost}")
        new_solution = tweak_0(current_solution)
        new_cost = get_cost(problem, new_solution)
        if new_cost >= current_cost:
            current_solution = new_solution
            current_cost = new_cost
            #print(f"  Nuovo miglioramento trovato: {current_cost}")
            
    return current_solution

$\beta$ = 1

In [343]:
def tweak_1(solution: list[int]) -> list[int]:
    solution_copy = solution.copy()
    path = [x for x in solution_copy if x!=0]
    length = len(path)
    if random.random() < 0.5:
        # 2-opt
        index1, index2 = random.sample(range(length), 2)
        start, end = min(index1, index2), max(index1, index2)
        path[start:end+1] = path[start:end+1][::-1]
    min_numbers_of_zeroes = int(length * 0.25) # at least 25% of the elements should be zeroes
    max_numbers_of_zeroes = int(length * 0.4)  # at most 40% of the elements should be zeroes

    number_of_zeroes_to_insert = random.randint(min_numbers_of_zeroes, max_numbers_of_zeroes)
    possible_positions = range(length-1)
    positions_to_insert_zeroes = set(random.sample(possible_positions, number_of_zeroes_to_insert))

    new_path = []
    for i, node in enumerate(path):
        new_path.append(node)
        if i in positions_to_insert_zeroes:
            new_path.append(0)
    return new_path

def hc_beta_equal_1(problem:Problem, max_iterations:int)-> list[int]:
    current_solution = get_initial_solution(problem._graph.nodes)
    current_cost = get_cost(problem, current_solution)
    
    #print(f"Starting Cost: {current_cost}")

    for i in range(max_iterations):
        #print(f"Iteration {i}: Actual Cost = {current_cost}")
        new_solution = tweak_1(current_solution)
        new_cost = get_cost(problem, new_solution)
        if new_cost < current_cost:
            current_solution = new_solution
            current_cost = new_cost
            #print(f"  Nuovo miglioramento trovato: {current_cost}")
            
    return current_solution

$\beta$ > 1

In [344]:
def tweak_2(solution: list[int]) -> list[int]:
    solution_copy = solution.copy()
    path = [x for x in solution_copy if x!=0]
    length = len(path)
    if random.random() < 0.5:
        # 2-opt
        index1, index2 = random.sample(range(length), 2)
        start, end = min(index1, index2), max(index1, index2)
        path[start:end+1] = path[start:end+1][::-1]
    
    min_numbers_of_zeroes = int(length * 0.5) # at least 50% of the elements should be zeroes
    max_numbers_of_zeroes = int(length * 0.99)  # at most 99% of the elements should be zeroes

    number_of_zeroes_to_insert = random.randint(min_numbers_of_zeroes, max_numbers_of_zeroes)
    possible_positions = range(length-1)
    positions_to_insert_zeroes = set(random.sample(possible_positions, number_of_zeroes_to_insert))

    new_path = []
    for i, node in enumerate(path):
        new_path.append(node)
        if i in positions_to_insert_zeroes:
            new_path.append(0)
    return new_path

def hc_beta_greater_than_1(problem:Problem, max_iterations:int)-> list[int]:
    current_solution = get_initial_solution(problem._graph.nodes)
    current_cost = get_cost(problem, current_solution)
    
    #print(f"Starting Cost: {current_cost}")

    for i in range(max_iterations):
        #print(f"Iteration {i}: Actual Cost = {current_cost}")
        new_solution = tweak_2(current_solution)
        new_cost = get_cost(problem, new_solution)
        if new_cost < current_cost:
            current_solution = new_solution
            current_cost = new_cost
            #print(f"  Nuovo miglioramento trovato: {current_cost}")
            
    return current_solution

===========================================================================================
### Funzione di Inizializzazione della Popolazione
===========================================================================================

In [345]:
def get_random_individual(problem:Problem) -> list[int]:
    # returned individual does not contain zeroes in the start or end 
    nodes = list(problem._graph.nodes)
    if problem._beta < 1.0:
        del(nodes[0])  # remove depot
        random.shuffle(nodes)
        return nodes 
    elif problem._beta == 1.0:
        random.shuffle(nodes)
        while nodes[0] != 0 and nodes[-1] != 0:
            random.shuffle(nodes)
        return nodes
    else:
        del(nodes[0])  # remove depot
        random.shuffle(nodes)
        number_of_zeroes = random.randint(int(len(nodes)*0.25), int(len(nodes)*0.99))
        possible_positions = range(len(nodes)-1)
        positions_to_insert_zeroes = set(random.sample(possible_positions, number_of_zeroes))
        path = []
        for i, node in enumerate(nodes):
            path.append(node)
            if i in positions_to_insert_zeroes:
                path.append(0)
        return path

In [346]:
def get_special_individual(problem:Problem)-> list[int]:
    if problem._beta < 1.0:
        return hc_beta_less_than_1(problem, max_iterations=1000)
    elif problem._beta == 1.0:
        return hc_beta_equal_1(problem, max_iterations=1000)
    else:
        return hc_beta_greater_than_1(problem, max_iterations=1000)
        
        

In [347]:
def two_opt(path: list[int], max_iterations:int) -> list[int]:
    best_path = list(path)
    length = len(best_path)
    for _ in range(max_iterations):
        i = random.randint(0, length - 2)
        j = random.randint(i + 1, length - 1)
        A = best_path[i - 1] if i > 0 else 0
        B = best_path[i]
        C = best_path[j]
        D = best_path[j + 1] if j < length - 1 else 0

        current_distance = DISTANCE_MATRIX[A][B] + DISTANCE_MATRIX[C][D]
        new_distance = DISTANCE_MATRIX[A][C] + DISTANCE_MATRIX[B][D]
        if new_distance < current_distance:
            best_path[i:j + 1] = best_path[i:j + 1][::-1]
    return best_path

def get_tsp_sequence(problem:Problem) -> list[int]:
    nodes = list(problem._graph.nodes)
    if 0 in nodes:
        nodes.remove(0)
    current_node = 0
    path = []
    unvisited_nodes = set(nodes)
    while unvisited_nodes:
        next_node = min(unvisited_nodes, key=lambda node: DISTANCE_MATRIX[current_node][node])
        path.append(next_node)
        unvisited_nodes.remove(next_node)
        current_node = next_node
    return two_opt(path, max_iterations=500)

def get_elite_for_beta_equal_one(problem:Problem, solution:list[int]) -> list[int]:
    path = solution.copy()
    beta = problem._beta
    alpha = problem._alpha

    current_path = []
    current_weight = 0.0

    for next_node in path:
        current_node = current_path[-1] if current_path else 0
        gold_at_next_node = problem._graph.nodes[next_node]['gold']
        # Cost if we go to next_node now
        dist_to_next_node = DISTANCE_MATRIX[current_node][next_node]
        weight_if_visited = current_weight + gold_at_next_node
        cost_to_next_node = dist_to_next_node + (alpha * dist_to_next_node * weight_if_visited) ** beta
        # Cost if we return to depot first
        dist_to_depot = DISTANCE_MATRIX[current_node][0]
        cost_to_depot = dist_to_depot + (alpha * dist_to_depot * current_weight) ** beta
        dist_depot_to_next = DISTANCE_MATRIX[0][next_node]
        cost_depot_to_next = dist_depot_to_next + (alpha * dist_depot_to_next * gold_at_next_node) ** beta
        total_cost_via_depot = cost_to_depot + cost_depot_to_next

        if total_cost_via_depot < cost_to_next_node:
            current_path.append(0)  # return to depot
            current_path.append(next_node)
            current_weight = gold_at_next_node
        else:
            current_path.append(next_node)
            current_weight += gold_at_next_node
    return current_path

def get_baseline_individual(solution:list[int]) -> list[int]:
    path = solution.copy()
    baseline_path = []
    for node in path:
        baseline_path.append(node)
        baseline_path.append(0)  
    baseline_path.pop() 
    return baseline_path

def get_elite_individual(problem:Problem) -> list[int]:
    path = get_tsp_sequence(problem)
    
    if problem._beta < 1.0:
        return path
    elif problem._beta == 1.0:
        return get_elite_for_beta_equal_one(problem, path)
    else:
        return get_baseline_individual(path)


In [348]:
def initialize_population(problem: Problem, population_size: int) -> list[list[int]]:
    population = []
    actual_population_size = 0
    population.append(get_elite_individual(problem))
    population.append(get_baseline_individual(get_tsp_sequence(problem)))
    actual_population_size= actual_population_size+2
    while actual_population_size < (int(population_size / 2)-1):
        population.append(get_special_individual(problem))
        actual_population_size += 1
    while actual_population_size <= (int(population_size / 2)-2):
        population.append(get_random_individual(problem))
        actual_population_size += 1    
    return population


===========================================================================================
### ALGORITMO GENETICO
===========================================================================================

$\beta$ < 1

In [349]:
def crossover_beta_less_than_one(parent1: list[int], parent2: list[int]) -> list[int]:
    size = len(parent1)
    start, end = sorted(random.sample(range(size), 2))
    child_p1 = parent1[start:end + 1]
    child_p1_set = set(child_p1)
    child_p2 = [item for item in parent2 if item not in child_p1_set]
    child = child_p2[:start] + child_p1 + child_p2[start:]
    return child

In [350]:
def mutation_beta_less_than_one(individual: list[int], mutation_rate: float) -> list[int]:
    if random.random() < mutation_rate:
        return individual
    individual_mutated = list(individual)
    length = len(individual_mutated)
    index1, index2 = random.sample(range(length), 2)
    start, end = min(index1, index2), max(index1, index2)
    individual_mutated[start:end+1] = individual_mutated[start:end+1][::-1]
    return individual_mutated

$\beta$ = 1

In [351]:
# SCAMBIA NODI NON ZERO

def mutation_beta_equal_one_OLDVERSION(individual:list[int], mutation_rate: float) -> list[int]:
    if random.random() < mutation_rate:
        return individual
    individual_mutated = list(individual)
    indexes = range(len(individual_mutated))
    nodes_only = [x for x in indexes if individual_mutated[x]!=0]
    index1, index2 = random.sample(nodes_only, 2)
    individual_mutated[index1], individual_mutated[index2] = individual_mutated[index2], individual_mutated[index1]
    return individual_mutated
    
    

In [352]:
def mutation_beta_equal_one(individual: list[int], mutation_rate: float) -> list[int]:
    if random.random() > mutation_rate:
        return individual
    individual_mutated = list(individual)


    if random.random() < 0.5:
        node_indices = [i for i,x in enumerate(individual_mutated) if x!=0]
        index1,index2 = sorted(random.sample(range(len(node_indices)), 2))
        nodes = [individual_mutated[i] for i in node_indices]
        nodes[index1:index2+1] = nodes[index1:index2+1][::-1]
        for i, node_index in enumerate(node_indices):
            individual_mutated[node_index] = nodes[i]
    else:
        zero_indices = [i for i, x in enumerate(individual_mutated) if x == 0]
        if zero_indices:
            random_zero_index = random.choice(zero_indices)
            random_direction = random.choice([-1, 1])
            swap_index = random_zero_index + random_direction
            individual_mutated[random_zero_index], individual_mutated[swap_index] = individual_mutated[swap_index], individual_mutated[random_zero_index]
    return individual_mutated

In [353]:
def crossover_beta_equal_one(parent1: list[int], parent2: list[int], cost1:float, cost2:float) -> list[int]:
    if cost1 <= cost2:
        better_parent = parent1
        other_parent = parent2
    else:
        better_parent = parent2
        other_parent = parent1
    child_mask = [x if x==0 else None for x in better_parent]
    nodes_source = [x for x in other_parent if x!=0]
    nodes_iterator = iter(nodes_source)
    child = []
    for node in child_mask:
        if node == 0:
            child.append(0)
        else:
            try:
                child.append(next(nodes_iterator))
            except StopIteration:
                break
    return child


==================================================================================================================

In [354]:
def get_offspring(problem:Problem, parent1:list[int], parent2:list[int], cost_p1:float, cost_p2:float) -> list[int]:
    if problem._beta < 1.0:
        if random.random() < 0.85:
            child = crossover_beta_less_than_one(parent1, parent2)
        elif cost_p1 < cost_p2:
            child = parent1.copy()
        else:
            child = parent2.copy()
        child = mutation_beta_less_than_one(child, mutation_rate=0.1)
        return child
    elif problem._beta == 1.0:
        if random.random() < 0.9:
            child = crossover_beta_equal_one(parent1, parent2, cost_p1, cost_p2)
        elif cost_p1 < cost_p2:
            child = parent1.copy()
        else:
            child = parent2.copy()
        child = mutation_beta_equal_one(child, mutation_rate=0.3)
        return child
    else:
        # Implement crossover and mutation for beta > 1.0
        pass

In [355]:
def selection_tournament(problem:Problem,population:list[list[int]], k=2)-> list[int]:
    competitors = random.sample(population, k)
    return min(competitors, key=lambda ind: get_cost(problem, ind))

In [356]:
def get_edges(individual:list[int]) -> set[tuple[int,int]]:
    edges = set()
    for i in range(len(individual) - 1):
        edge = tuple(sorted((individual[i], individual[i + 1])))
        edges.add(edge)
    return edges

def calculate_differences(individual_1:list[int], individual_2:list[int]) -> int:
    edges1 = get_edges(individual_1)
    edges2 = get_edges(individual_2)
    differences = edges1.symmetric_difference(edges2)
    return len(differences)

In [357]:
def get_new_generation(problem:Problem, current_population:list[list[int]]) -> list[list[int]]:
    population_size = len(current_population)
    random.shuffle(current_population)
    next_generation = []
    for i in range(0, population_size, 2):
        if i + 1 >= population_size:
            next_generation.append(current_population[i])
            break
            
        parent1 = current_population[i]
        parent2 = current_population[i+1]
        cost_parent1 = get_cost(problem, parent1)
        cost_parent2 = get_cost(problem, parent2)
        

        child_1 = get_offspring(problem, parent1, parent2, cost_parent1, cost_parent2)
        child_2 = get_offspring(problem, parent2, parent1, cost_parent2, cost_parent1) 
        
        cost_child_1 = get_cost(problem, child_1)
        cost_child_2 = get_cost(problem, child_2)
        
        difference_between_parent1_child1 = calculate_differences(parent1, child_1)
        difference_between_parent2_child2 = calculate_differences(parent2, child_2)
        
        difference_between_parent1_child2 = calculate_differences(parent1, child_2)
        difference_between_parent2_child1 = calculate_differences(parent2, child_1)
        
        if (difference_between_parent1_child1 + difference_between_parent2_child2) <= (difference_between_parent1_child2 + difference_between_parent2_child1):
            winner1 = child_1 if cost_child_1 < cost_parent1 else parent1
            winner2 = child_2 if cost_child_2 < cost_parent2 else parent2
        else:
            winner1 = child_2 if cost_child_2 < cost_parent1 else parent1
            winner2 = child_1 if cost_child_1 < cost_parent2 else parent2
            
        next_generation.append(winner1)
        next_generation.append(winner2)
        
    return next_generation

In [358]:
import time

def get_naive_baseline(problem: Problem) -> list[int]:
    nodes = list(problem.graph.nodes)
    if 0 in nodes: nodes.remove(0)
    nodes.sort() 
    
    baseline_path = [0]
    for node in nodes:
        baseline_path.append(node)
        baseline_path.append(0)
        
    return baseline_path

def genetic_algorithm(problem: Problem, 
                      pop_size: int = 50, 
                      max_generations: int = 1000) -> tuple[list[int], float]:
    start_time = time.time()
    
    print(f"\n{'='*40}")
    print(f"   AVVIO GENETIC ALGORITHM (Beta={problem._beta})")
    print(f"{'='*40}")
    
    # 1. Calcolo Baseline e Inizializzazione
    baseline_sol = get_naive_baseline(problem)
    baseline_cost = get_cost(problem, baseline_sol)
    print(f"[*] Costo Baseline (Naive): {baseline_cost:.2f}")
    
    population = initialize_population(problem, pop_size)
    
    # Troviamo il migliore iniziale
    global_best_solution = min(population, key=lambda ind: get_cost(problem, ind))
    global_best_cost = get_cost(problem, global_best_solution)
    
    print(f"[*] Costo Migliore Iniziale (Elite): {global_best_cost:.2f}")
    start_improvement = 100 * (baseline_cost - global_best_cost) / baseline_cost
    print(f"[*] Miglioramento iniziale su Baseline: +{start_improvement:.2f}%")
    print(f"{'-'*40}")

    # 2. Ciclo Evolutivo (Senza limite di tempo)
    for generation in range(1, max_generations + 1):
        
        # A. Creazione Nuova Generazione (Deterministic Crowding)
        population = get_new_generation(problem, population)
        
        # B. Monitoraggio e Aggiornamento Best
        current_best_ind = min(population, key=lambda ind: get_cost(problem, ind))
        current_best_cost = get_cost(problem, current_best_ind)
        
        if current_best_cost < global_best_cost:
            global_best_cost = current_best_cost
            global_best_solution = list(current_best_ind)
            print(f"Gen {generation}: Nuovo Record! {global_best_cost:.2f}")
            
        # Log periodico
        if generation % 100 == 0:
            print(f"   ... Gen {generation} completata. Best: {global_best_cost:.2f}")

    # 3. Report Finale e Confronto
    total_time = time.time() - start_time
    final_improvement = 100 * (baseline_cost - global_best_cost) / baseline_cost
    
    print(f"\n{'='*40}")
    print(f"   RISULTATI FINALI")
    print(f"{'='*40}")
    print(f"Generazioni Totali : {max_generations}")
    print(f"Tempo di Esecuzione: {total_time:.2f}s")
    print(f"{'-'*40}")
    print(f"Costo Baseline     : {baseline_cost:.2f}")
    print(f"Costo GA Ottimale  : {global_best_cost:.2f}")
    print(f"Risparmio Assoluto : {baseline_cost - global_best_cost:.2f}")
    print(f"Miglioramento %    : {final_improvement:.2f}%")
    print(f"{'='*40}\n")
    
    return global_best_solution, global_best_cost

===========================================================================================

In [359]:

genetic_algorithm(problem, pop_size=100, max_generations=3000)


   AVVIO GENETIC ALGORITHM (Beta=1)
[*] Costo Baseline (Naive): 18772.91
[*] Costo Migliore Iniziale (Elite): 18772.91
[*] Miglioramento iniziale su Baseline: +0.00%
----------------------------------------
Gen 1: Nuovo Record! 18772.91
Gen 7: Nuovo Record! 18772.70
Gen 8: Nuovo Record! 18772.67
Gen 10: Nuovo Record! 18772.56
Gen 28: Nuovo Record! 18772.49
   ... Gen 100 completata. Best: 18772.49
Gen 112: Nuovo Record! 18772.11
Gen 141: Nuovo Record! 18771.72
Gen 191: Nuovo Record! 18771.23
   ... Gen 200 completata. Best: 18771.23
Gen 207: Nuovo Record! 18771.23
   ... Gen 300 completata. Best: 18771.23
Gen 345: Nuovo Record! 18770.86
   ... Gen 400 completata. Best: 18770.86
   ... Gen 500 completata. Best: 18770.86
   ... Gen 600 completata. Best: 18770.86
Gen 666: Nuovo Record! 18770.86
   ... Gen 700 completata. Best: 18770.86
Gen 722: Nuovo Record! 18770.67
Gen 723: Nuovo Record! 18770.67
Gen 797: Nuovo Record! 18770.67
   ... Gen 800 completata. Best: 18770.67
Gen 893: Nuovo R

([28,
  0,
  47,
  0,
  85,
  0,
  56,
  0,
  61,
  0,
  0,
  62,
  4,
  18,
  0,
  0,
  16,
  0,
  53,
  0,
  23,
  0,
  55,
  0,
  72,
  0,
  7,
  0,
  46,
  0,
  25,
  0,
  93,
  0,
  74,
  0,
  67,
  0,
  44,
  0,
  38,
  0,
  0,
  45,
  14,
  0,
  19,
  0,
  78,
  0,
  73,
  0,
  70,
  0,
  66,
  0,
  37,
  0,
  29,
  0,
  42,
  0,
  24,
  0,
  35,
  0,
  97,
  0,
  81,
  0,
  0,
  41,
  12,
  0,
  0,
  95,
  10,
  0,
  27,
  0,
  84,
  0,
  49,
  0,
  77,
  0,
  64,
  0,
  80,
  0,
  51,
  0,
  20,
  0,
  31,
  0,
  30,
  0,
  21,
  0,
  33,
  0,
  1,
  0,
  11,
  0,
  6,
  0,
  75,
  0,
  43,
  0,
  92,
  0,
  26,
  0,
  90,
  0,
  36,
  0,
  94,
  0,
  17,
  0,
  59,
  0,
  32,
  0,
  88,
  0,
  60,
  0,
  40,
  0,
  69,
  0,
  8,
  0,
  63,
  0,
  48,
  0,
  91,
  0,
  82,
  0,
  39,
  0,
  15,
  0,
  98,
  0,
  5,
  0,
  86,
  0,
  0,
  2,
  57,
  0,
  65,
  0,
  54,
  0,
  83,
  0,
  50,
  0,
  89,
  0,
  3,
  0,
  79,
  0,
  22,
  0,
  58,
  0,
  52,
  9,
  0,
  0,
  96,
  